# Mode choice calibration 

- Read original coeff csv file
- Change +- 20% of coefficients 
- Save change in a new csv file
- Update yaml file with new file name

In [1]:
import pandas as pd 
import yaml

In [2]:
coeffs = pd.read_csv('configs/configs/tour_mode_choice_coeffs.csv', 
                     index_col = 'Expression')

coeff_t = coeffs.drop(columns= 'atwork').T
at_work_coeff = coeffs['atwork'].reset_index(drop = True)

In [3]:
# Transit IVT: Modify coefficient by 20% (This will make the coefficient more negative)
# THIS ARE NOT likely to change model results
c_ivt_coeff = ['c_ivt_lrt', 'c_ivt_ferry', 'c_ivt_exp','c_ivt_hvy', 'c_ivt_com']
for col_name in c_ivt_coeff:
    # Split str colum in float and str part. 
    split = coeff_t[col_name].str.split('*', expand = True) 
    split[0] = (split[0].astype(float)*1.2).astype(str)
    new_col= split[0] + '*' + split[1] #Bri
    coeff_t[col_name] = new_col

In [4]:
# Modify constants 
# Walk ASC 
walk_asc = ['walk_ASC_no_auto', 'walk_ASC_auto_decicient', 'walk_ASC_auto_sufficient',
            'joint_walk_ASC_no_auto', 'joint_walk_ASC_auto_decicient', 
            'joint_walk_ASC_auto_sufficient', 'c_ivt']
for col_name in walk_asc:
    col = coeff_t[col_name].astype(float)
    col.where(col<0,col*2, inplace = True) # change this value to 1.2 (This was just right)
    col.where(col>0,col*0.8, inplace = True)
    coeff_t[col_name] = col
    
# Walk and drive to transit constants
walk_drive_transit_asc = ['walk_transit_ASC_no_auto', 
                          'walk_transit_ASC_auto_decicient',
                          'walk_transit_ASC_auto_sufficient', 
                          'drive_transit_ASC_auto_decicient',
                          'drive_transit_ASC_auto_sufficient']

for col_name in walk_drive_transit_asc:
    col = coeff_t[col_name].astype(float)
    col.where(col>0,col*1.2, inplace = True)
    col.where(col<0,col*0.8, inplace = True)
    coeff_t[col_name] = col

In [5]:
# Transit IVT: Modify coefficient by 20% (This will make the coefficient more negative)
c_ivt_coeff = ['c_walktimeshort', 'c_walktimelong']
for col_name in c_ivt_coeff:
    # Split str colum in float and str part. 
    split = coeff_t[col_name].str.split('*', expand = True) 
    split[0] = (split[0].astype(float)*0.8).astype(str)
    new_col= split[0] + '*' + split[1] #Bri
    coeff_t[col_name] = new_col

In [6]:
# coeff_t['atwork'] = at_work_coeff
coeff_final = coeff_t.T.reset_index()
coeff_final['atwork'] = at_work_coeff

In [7]:
new_name = 'tour_mode_choice_coeffs_v2.csv'
coeff_final.to_csv('configs/configs/'+ new_name, index = False)

## Modify YAML file

In [8]:
yaml_file = open('configs/configs/tour_mode_choice.yaml')
model_settings = yaml.load(yaml_file, Loader=yaml.FullLoader)

In [9]:
# Change CostPerMile 
model_settings['CONSTANTS']['costPerMile'] = 18.29 * 0.1 #Reduce cost per mile
model_settings['COEFFS'] = new_name

In [10]:
with open('configs/configs/tour_mode_choice.yaml', 'w') as file:
    documents = yaml.dump(model_settings, file)